In [1]:
import sys
sys.path.append("../../Libs/")
from AP_class import APDFT_perturbator as AP
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np
from numpy.linalg import norm as norm
from scipy.optimize import  minimize
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [2]:
%load_ext autoreload
%autoreload 2
from AP_class import APDFT_perturbator as AP
from ABSE import absec
from Morse import Morse_interpolator

In [3]:
from berny import Berny, geomlib, coords,Geometry,Math
import berny

In [4]:
def anglecenter(coords):
    return np.around(np.arccos(coords[1].dot(coords[2])/norm(coords[1])/norm(coords[2]))*180/np.pi,decimals=3)

In [5]:
def build_h_ic(s,g_ic,h0,B_inv):
    geom0=s.geom.copy()
    B=s.coords.B_matrix(geom0)
    bms=[]
    for i in range(geom0.coords.flatten().shape[0]):
        a=geom0.coords.flatten()
        geom1=geom0.copy()
        a[i]+=.001*bohr2ang
        a=a.reshape(geom0.coords.shape)
        geom1.coords=a
        bms.append((s.coords.B_matrix(geom1)-B)*1000)
    bms_arr=np.asarray(bms)
    BpG2=np.einsum('ijk,j->ik',bms,g_ic)
    h_ic=B_inv.T@(h0-BpG2)@B_inv
    return h_ic


ebde=100/627.5 # empirical bond dissociation energy
ch4coords,nh3optg,oh2optg,hfoptg=np.load("optgeoms.npy",allow_pickle=True)
def mpa_pb_with_energy(coords,atoml,g,h,e,gic=False,ghost=[]):
    g=np.asarray(g)
    if not len(h.shape)==2:
        h=h.swapaxes(1,2)
        h=h.reshape(g.shape[0]*3,g.shape[0]*3)
    geom0=Geometry(atoml,coords*bohr2ang)
    bernyobj=Berny(geom0)
    s=bernyobj._state
    B = s.coords.B_matrix(geom0)
    q0=s.coords.eval_geom(geom0)
    B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))
    if not gic:
        g_ic=np.dot(B_inv.T, (g).reshape(-1))
    else:
        g_ic=g
    h_ic=build_h_ic(s,g_ic,h,B_inv)
    bnr=0
    dde_mb=0
    ddq_mb=np.zeros_like(q0)
    for i in range(len(s.coords._coords)):
        if s.coords._coords[i].__class__ is berny.coords.Bond:
            bnr+=1
            if s.coords._coords[i].i not in ghost and s.coords._coords[i].j not in ghost:
                MorseInt=Morse_interpolator(g_ic[i],h_ic[i,i],q0[i],ebde,e)
                ddq_mb[i]+=MorseInt.re+g_ic[i]/h_ic[i,i]-q0[i]
                dde_mb+=MorseInt.e_min-e+1/2*h_ic[i,i]*(MorseInt.re-q0[i])**2
    dq_NR=-Math.pinv(h_ic)@g_ic
    ddq_mb_old=np.linalg.solve(h_ic[bnr:,bnr:],-h_ic[bnr:,:]@(ddq_mb))
    ddq_mb[bnr:]=-Math.pinv(h_ic[bnr:,bnr:]) @(h_ic[bnr:,:]@(ddq_mb))
    dq_tot=dq_NR+ddq_mb
    en_MB=(e-1/2*np.einsum("i,ij,j",dq_tot,h_ic,dq_tot)+dde_mb)
    print (s.coords._coords)
    return q0,q0+dq_tot,en_MB

In [6]:
HF=gto.M(atom="F 0 0 0;H 0 0 1.695741",unit="bohr",
            basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])})
mfhf = scf.RHF(HF)
mfhf.scf()

converged SCF energy = -100.065540168732


-100.0655401687325

In [7]:
dm=mfhf.make_rdm1()

In [8]:
def Vne(coords,mol=HF,dm=dm):
    mol.set_rinv_orig_(coords)
    dV=mol.intor('int1e_rinv')
    return -np.einsum('ij,ij',dV,dm)

In [9]:
dL=[-1,0,0,0] #-1 charge central atom
def Vnn(coords,mol=HF):
    Vnn=0
    for i in range (mol.natm):
        #Vnn+= (mol.atom_charge(i))/np.linalg.norm(coords-mol.atom_coord(i))        
        Vnn+= (mol.atom_charge(i)+dL[i])/np.linalg.norm(coords-mol.atom_coord(i))
    return Vnn

In [10]:
def Vtot(coords,mol=HF,dm=dm):
    return Vnn(coords,mol)+Vne(coords,mol,dm)

In [11]:
def euler2coords(teta,phi,r):
    return np.asarray([np.cos(phi)*np.sin(teta),np.sin(teta)*np.sin(phi),np.cos(teta)])*r
def Vtot_fl(angles,r,mol=HF,dm=dm):
    teta,phi=angles
    coords=euler2coords(teta,phi,1.695741)
    return Vtot(coords,mol=HF,dm=dm)

In [12]:
prot_site=minimize(Vtot,[0,.5,-1.3]).x
norm(prot_site),anglecenter(np.vstack([HF.atom_coords(),prot_site])) 

(1.5510758818441097, 118.422)

In [13]:
pHF=FcM(fcs=[0,0,-1],atom="F 0 0 0;H 0 0 1.695741; H {} {} {}".format(prot_site[0],prot_site[1],prot_site[2]),unit="bohr",
          basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])},charge=1)
pHF.charge=0

In [14]:
pHFmf=scf.RHF(pHF)
pHFmf.scf(dm0=pHFmf.init_guess_by_1e())

converged SCF energy = -100.065874806313


-100.06587480631325

In [15]:
afHF=aaff(pHFmf,[-1,0,1])+alc_deriv_grad_nuc(pHF,[-1,0,1])
apHF=AP(pHFmf,sites=[0,2])
apHF.build_all()
apHF.APDFT3([-1,1])

-76.01756091477334

In [16]:
pHFm=FcM(fcs=[.1,0,-1.1],atom="F 0 0 0;H 0 0 1.695741; H {} {} {}".format(prot_site[0],prot_site[1],prot_site[2]),unit="bohr",
          basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])},charge=1)
pHFmfm=scf.RHF(pHFm)
pHFmfm.scf(dm0=pHFmfm.init_guess_by_1e())

converged SCF energy = -102.756154130817


-102.75615413081657

In [17]:
afHFm=aaff(pHFmfm,[-1,0,1])+alc_deriv_grad_nuc(pHFm,[-1,0,1])
apHFm=AP(pHFmfm,sites=[0,2])
apHFm.build_all()
apHFm.APDFT3([-1,1])

-78.18122890422204

In [18]:
pHFp=FcM(fcs=[-.1,0,-.9],atom="F 0 0 0;H 0 0 1.695741; H {} {} {}".format(prot_site[0],prot_site[1],prot_site[2]),unit="bohr",
          basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])},charge=1)
pHFmfp=scf.RHF(pHFp)
pHFmfp.scf(dm0=pHFmfp.init_guess_by_1e())

converged SCF energy = -97.4269980312757


-97.42699803127572

In [19]:
afHFp=aaff(pHFmfp,[-1,0,1])+alc_deriv_grad_nuc(pHFp,[-1,0,1])
apHFp=AP(pHFmfp,sites=[0,2])
apHFp.build_all()

In [20]:
HFh0=pHFmf.Hessian().hess()
HFhm=pHFmfm.Hessian().hess()
HFhp=pHFmfp.Hessian().hess()

In [21]:
HFht=HFh0+(HFhp-HFhm)/.2+(HFhp+HFhm-2*HFh0)/2/.01

In [22]:
HFgt=afHF+(afHFp-afHFm)/.2/2+(afHFp+afHFm-2*afHF)/6/.01

In [23]:
HFet=apHF.APDFT3([-1,1])+absec("F","O")+(apHFp.third_deriv([-1,1])-apHFm.third_deriv([-1,1]))/.2/24+\
       (apHFp.third_deriv([-1,1])+apHFm.third_deriv([-1,1])-2*apHF.third_deriv([-1,1]))/.01/120                                  

In [24]:
mpa_pb_with_energy(pHF.atom_coords(),['O','H','H'],HFgt,HFht,HFet)


[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Angle(1, 0, 2, weak=0)]


(array([1.695741  , 1.55107588, 2.06684813]),
 array([1.77230706, 1.72836202, 1.87445854]),
 -76.05698670461706)

In [25]:
1.8744619*180/np.pi,apHF.APDFT3([-1,1]),HFet

(107.39875572807337, -76.01756091477334, -76.03880557464734)

In [26]:
absec("F","O"),absec("O","N"),absec("N","C")

(-0.0029667638878976277, -0.003269512295844379, -0.004133047217614205)

# H$_2$O

In [26]:
oh2lst=[]
for x in range(3):
    oh2lst.append("OHH"[x])
    oh2lst.extend(oh2optg[x].tolist())  
oh2srt=(("{} "*4+";")*3)[:-1].format(*oh2lst)

In [27]:
OH2=gto.M(atom=oh2srt,unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])})
mfoh2 = scf.RHF(OH2)
mfoh2.scf()
dmoh2=mfoh2.make_rdm1()

converged SCF energy = -76.0633193336666


In [55]:
ps_h2o=minimize(Vtot,np.array([0,0,1.5]),args=(OH2,dmoh2)).x

In [54]:
OH2.atom_coords()

array([[ 0.        ,  0.        ,  0.        ],
       [ 1.73352672, -0.38419879,  0.        ],
       [-0.12082414,  1.77147401,  0.        ]])

In [56]:
pH2O=FcM(fcs=[0,0,0,-1],atom=oh2srt+"; H {} {} {}".format(*ps_h2o),\
         unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])},charge=1)
pH2Omf=scf.RHF(pH2O)
pH2Omf.scf(pH2Omf.init_guess_by_1e())

converged SCF energy = -76.0638566148365


-76.0638566148365

In [57]:
pH2Op=FcM(fcs=[-.1,0,0,-.9],atom=oh2srt+"; H {} {} {}".format(*ps_h2o),\
         unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])},charge=1)
pH2Omfp=scf.RHF(pH2Op)
pH2Omfp.scf(pH2Omfp.init_guess_by_1e())

converged SCF energy = -73.8541070148404


-73.85410701484035

In [58]:
pH2Om=FcM(fcs=[.1,0,0,-1.1],atom=oh2srt+"; H {} {} {}".format(*ps_h2o),\
         unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])},charge=1)
pH2Omfm=scf.RHF(pH2Om)
pH2Omfm.scf(pH2Omfm.init_guess_by_1e())

converged SCF energy = -78.3219843321534


-78.32198433215336

In [59]:
afph2o=aaff(pH2Omf,[-1,0,0,1])+alc_deriv_grad_nuc(pH2O,[-1,0,0,1])
afph2op=aaff(pH2Omfp,[-1,0,0,1])+alc_deriv_grad_nuc(pH2Op,[-1,0,0,1])
afph2om=aaff(pH2Omfm,[-1,0,0,1])+alc_deriv_grad_nuc(pH2Om,[-1,0,0,1])

In [60]:
afph2ot=afph2o+(afph2op-afph2om)/.2/2+(afph2op+afph2om-2*afph2o)/6/.01

In [61]:
pH2Oh=pH2Omf.Hessian().hess()
pH2Ohp=pH2Omfp.Hessian().hess()
pH2Ohm=pH2Omfm.Hessian().hess()

In [62]:
pH2Oht=pH2Oh+(pH2Ohp-pH2Ohm)/.2+(pH2Ohp+pH2Ohm-2*pH2Oh)/2/.01

In [63]:
apH2O=AP(pH2Omf,sites=[0,3])
apH2O.build_all()
apH2Op=AP(pH2Omfp,sites=[0,3])
apH2Op.build_all()
apH2Om=AP(pH2Omfm,sites=[0,3])
apH2Om.build_all()

In [64]:
pH2Oet=apH2O.APDFT3([-1,1])+absec("O","N")+(apH2Op.third_deriv([-1,1])-apH2Om.third_deriv([-1,1]))/.2/24+\
       (apH2Op.third_deriv([-1,1])+apH2Om.third_deriv([-1,1])-2*apH2O.third_deriv([-1,1]))/.01/120                   

In [65]:
mpa_pb_with_energy(pH2O.atom_coords(),['N','H','H','H'],afph2ot,pH2Oht,pH2Oet)

[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Bond(0, 3, weak=0), Angle(1, 0, 2, weak=0), Angle(1, 0, 3, weak=0), Angle(2, 0, 3, weak=0)]


(array([1.77559105, 1.77558966, 1.61808131, 1.8569993 , 2.07365537,
        2.07365448]),
 array([1.88000882, 1.88000705, 1.83075698, 1.90961436, 1.90690986,
        1.90690844]),
 -56.217184647461266)

In [68]:
1.90961436*180/np.pi,1.90690986*180/np.pi

(109.4128433255758, 109.2578868898827)

In [41]:
1.90951409*180/np.pi,1.90686496*180/np.pi

(109.40709827776404, 109.25531430938256)

#     NH$_3$

In [42]:
ch4optg,nh3optg,oh2optg,fhoptg=np.load("optgeoms.npy",allow_pickle=True)
nh3lst=[]
for x in range(4):
    nh3lst.append("NHHH"[x])
    nh3lst.extend(nh3optg[x].tolist())
nh3srt=(("{} "*4+";")*4)[:-1].format(*nh3lst)
NH3=gto.M(atom=nh3srt,unit="bohr",basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])})
mf3 = scf.RHF(NH3)
mf3.scf()

converged SCF energy = -56.2221836357521


-56.22218363575205

In [43]:
nh3dm=mf3.make_rdm1()
ps_nh3=minimize(Vtot,np.array([0,-1.5,.3]),args=(NH3,nh3dm)).x

In [44]:
ps_nh3

array([-0.83061461, -0.09801929, -1.50548167])

In [45]:
pNH3=FcM(fcs=[0,0,0,0,-1],atom=nh3srt+"; H {} {} {} ".format(*ps_nh3),unit="bohr",\
         basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])},charge=1)
pNH3p=FcM(fcs=[-.1,0,0,0,-.9],atom=nh3srt+"; H {} {} {} ".format(*ps_nh3),unit="bohr",\
         basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])},charge=1)
pNH3m=FcM(fcs=[.1,0,0,0,-1.1],atom=nh3srt+"; H {} {} {} ".format(*ps_nh3),unit="bohr",\
         basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])},charge=1)
pNH3mf=scf.RHF(pNH3)
pNH3mf.scf(dm0=pNH3mf.init_guess_by_1e())
pNH3mfm=scf.RHF(pNH3m)
pNH3mfm.scf(dm0=pNH3mfm.init_guess_by_1e())
pNH3mfp=scf.RHF(pNH3p)
pNH3mfp.scf(dm0=pNH3mfp.init_guess_by_1e())

converged SCF energy = -56.2226233985117
converged SCF energy = -58.0803727321716
converged SCF energy = -54.4098977434469


-54.40989774344688

In [46]:
NH3h=pNH3mf.Hessian().hess()
NH3hp=pNH3mfp.Hessian().hess()
NH3hm=pNH3mfm.Hessian().hess()

In [47]:
NH3ht=NH3h+(NH3hp-NH3hm)/.2+(NH3hp+NH3hm-2*NH3h)/.01/2

In [48]:
apNH3=AP(pNH3mf,sites=[0,4])
apNH3.build_all()
apNH3p=AP(pNH3mfp,sites=[0,4])
apNH3p.build_all()
apNH3m=AP(pNH3mfm,sites=[0,4])
apNH3m.build_all()

In [49]:
NH3et=apNH3.APDFT3([-1,1])+absec("N","C")+(apNH3p.third_deriv([-1,1])-apNH3m.third_deriv([-1,1]))/.2/24+\
    (apNH3p.third_deriv([-1,1])+apNH3m.third_deriv([-1,1])-2*apNH3.third_deriv([-1,1]))/120/.01

In [50]:
afpnh3=aaff(pNH3mf,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3,[-1,0,0,0,1])
afpnh3p=aaff(pNH3mfp,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3p,[-1,0,0,0,1])
afpnh3m=aaff(pNH3mfm,[-1,0,0,0,1])+alc_deriv_grad_nuc(pNH3m,[-1,0,0,0,1])

In [51]:
NH3gt=afpnh3+(afpnh3p-afpnh3m)/.2/2+(afpnh3p+afpnh3m-2*afpnh3)/.01/6

In [52]:
mpa_pb_with_energy(pNH3.atom_coords(),['C','H','H','H','H'],NH3gt,NH3ht,NH3et)

[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Bond(0, 3, weak=0), Bond(0, 4, weak=0), Angle(1, 0, 2, weak=0), Angle(1, 0, 3, weak=0), Angle(1, 0, 4, weak=0), Angle(2, 0, 3, weak=0), Angle(2, 0, 4, weak=0), Angle(3, 0, 4, weak=0)]


(array([1.88552043, 1.8855208 , 1.88552075, 1.7222089 , 1.88869543,
        1.88869544, 1.93209454, 1.88868181, 1.93212302, 1.93206923]),
 array([2.04782166, 2.04782373, 2.04782299, 1.97750663, 1.9090081 ,
        1.90900516, 1.91265353, 1.90898015, 1.91264891, 1.91264958]),
 -40.21285874630084)

In [53]:
1.90900809*180/np.pi,1.91265339*180/np.pi

(109.37810661333042, 109.58696691838946)